In [1]:
'''
@Author: Stefan Angelov
@Created: 04/23/2021
@Purpose: Predicting the air pollution
'''

'\n@Author: Stefan Angelov\n@Created: 04/23/2021\n@Purpose: Predicting the air pollution\n'

In [3]:
from src.data_preprocessing import *
from src.model_training import *

import warnings
warnings.filterwarnings('ignore')

# Data Cleaning and Pre Processing

In [4]:
df = load_dataset()
df = datetime_index_column(df)
df = df[wanted_col]
cleaned_df = df.interpolate()
del df
df_list = data_frames(cleaned_df)

df_PM25, df_PM10, df_SO2, df_NO2, df_CO, df_O3 = resample_data(df_list)
dfm_PM25, dfm_PM10, dfm_SO2, dfm_NO2, dfm_CO, dfm_O3 = resample_data(df_list, freq = 'MS')

df_list = [df_PM25, df_PM10, df_SO2, df_NO2, df_CO, df_O3]
dfm_list = [dfm_PM25, dfm_PM10, dfm_SO2, dfm_NO2, dfm_CO, dfm_O3]

dtrain_test_list = train_test_split(df_list)
mtrain_test_list = train_test_split(dfm_list)

save_train_test_data(dtrain_test_list, dtrain_dir, dtest_dir)
save_train_test_data(mtrain_test_list, mtrain_dir, mtest_dir)

   No  year  month  day  hour  PM2.5  PM10   SO2   NO2     CO    O3  TEMP  \
0   1  2013      3    1     0    4.0   4.0   4.0   7.0  300.0  77.0  -0.7   
1   2  2013      3    1     1    8.0   8.0   4.0   7.0  300.0  77.0  -1.1   
2   3  2013      3    1     2    7.0   7.0   5.0  10.0  300.0  73.0  -1.1   
3   4  2013      3    1     3    6.0   6.0  11.0  11.0  300.0  72.0  -1.4   
4   5  2013      3    1     4    3.0   3.0  12.0  12.0  300.0  72.0  -2.0   

     PRES  DEWP  RAIN   wd  WSPM       station  
0  1023.0 -18.8   0.0  NNW   4.4  Aotizhongxin  
1  1023.2 -18.2   0.0    N   4.7  Aotizhongxin  
2  1023.5 -18.2   0.0  NNW   5.6  Aotizhongxin  
3  1024.5 -19.4   0.0   NW   3.1  Aotizhongxin  
4  1025.2 -19.5   0.0    N   2.0  Aotizhongxin  


# LSTM Univariate Time Series Forecasting Model

In [7]:
train_PM25 = pd.read_csv('dataset/daily/train/train_PM2.5.csv', parse_dates = [0], index_col = [0])
test_PM25 = pd.read_csv('dataset/daily/test/test_PM2.5.csv', parse_dates = [0], index_col = [0])

scaled_train_PM25, scaler = transform_data(train_PM25)

time_steps = 365
n_feature = 1
model = fit_LSTM(scaled_train_PM25, time_steps, n_feature)


In [ ]:
predictions = predict_LSTM(scaled_train_PM25, scaler, train_PM25, test_PM25,time_steps, n_feature, model)

# LSTM Multivariate Time Series Forecasting Model

In [6]:
train_PM25 = pd.read_csv('dataset/daily/train/train_PM2.5.csv', parse_dates = [0], index_col = [0])
test_PM25 = pd.read_csv('dataset/daily/test/test_PM2.5.csv', parse_dates = [0], index_col = [0])
train_PM10 = pd.read_csv('dataset/daily/train/train_PM10.csv', parse_dates = [0], index_col = [0])
test_PM10 = pd.read_csv('dataset/daily/test/test_PM10.csv', parse_dates = [0], index_col = [0])
train_SO2 = pd.read_csv('dataset/daily/train/train_SO2.csv', parse_dates = [0], index_col = [0])
test_SO2 = pd.read_csv('dataset/daily/test/test_SO2.csv', parse_dates = [0], index_col = [0])
train_NO2 = pd.read_csv('dataset/daily/train/train_NO2.csv', parse_dates = [0], index_col = [0])
test_NO2 = pd.read_csv('dataset/daily/test/test_NO2.csv', parse_dates = [0], index_col = [0])
train_CO = pd.read_csv('dataset/daily/train/train_CO.csv', parse_dates = [0], index_col = [0])
test_CO = pd.read_csv('dataset/daily/test/test_CO.csv', parse_dates = [0], index_col = [0])
train_O3 = pd.read_csv('dataset/daily/train/train_O3.csv', parse_dates = [0], index_col = [0])
test_O3 = pd.read_csv('dataset/daily/test/test_O3.csv', parse_dates = [0], index_col = [0])

list_test = [test_PM25, test_PM10, test_SO2, test_NO2, test_CO, test_O3]
list_train = [train_PM25, train_PM10, train_SO2, train_NO2, train_CO, train_O3]

scaled_train_PM25, scaler = transform_data(train_PM25)
scaled_train_PM10, scaler = transform_data(train_PM10)
scaled_train_SO2, scaler = transform_data(train_SO2)
scaled_train_NO2, scaler = transform_data(train_NO2)
scaled_train_CO, scaler = transform_data(train_CO)
scaled_train_PM10, scaler = transform_data(train_PM10)
scaled_train_O3, scaler = transform_data(train_O3)


list_scaled_train = [scaled_train_PM25, scaled_train_PM10, scaled_train_SO2, scaled_train_NO2,
 scaled_train_CO, scaled_train_O3]

all_attributes = []
joined_element = []

for i in range(0, len(scaled_train_O3)):
    for j in range(0, len(list_scaled_train)):
        joined_element = np.append(joined_element, list_scaled_train[j][i])
    all_attributes.append(joined_element)
    joined_element = []

time_steps = 365
n_feature = 6
model = fit_LSTM(np.array(all_attributes), time_steps, n_feature)

model.save('trained_model.h5', model)

# model = tf.keras.models.load_model('trained_model.h5')

Epoch 1/10
122/122 [==============================] - 37s 294ms/step - loss: 0.0303
Epoch 2/10
122/122 [==============================] - 37s 303ms/step - loss: 0.0170
Epoch 3/10
122/122 [==============================] - 36s 291ms/step - loss: 0.0172
Epoch 4/10
122/122 [==============================] - 36s 293ms/step - loss: 0.0154
Epoch 5/10
122/122 [==============================] - 36s 292ms/step - loss: 0.0147
Epoch 6/10
122/122 [==============================] - 37s 305ms/step - loss: 0.0156
Epoch 7/10
122/122 [==============================] - 37s 303ms/step - loss: 0.0144
Epoch 8/10
122/122 [==============================] - 37s 306ms/step - loss: 0.0134
Epoch 9/10
122/122 [==============================] - 37s 299ms/step - loss: 0.0112
Epoch 10/10
122/122 [==============================] - 38s 310ms/step - loss: 0.0162
